### Installing all the libraries

In [ ]:
import os
import re

import pandas as pd

# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

from langchain.docstore.document import Document

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import time

import os
# from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, LlavaForConditionalGeneration

model_path = "zzxslp/som-llava-v1.5-13b-hf"

processor = AutoProcessor.from_pretrained(model_path, use_fast=False)

model = LlavaForConditionalGeneration.from_pretrained(model_path)



Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
import torch

# Move model to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Prepare prompt and image
prompt = "USER: <image>\nWhat's the content of the image? ASSISTANT:"
url = "https://www.ilankelman.org/stopsigns/australia.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# Process inputs and move tensors to GPU
inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)

# Generate response using GPU
with torch.no_grad():
    generate_ids = model.generate(**inputs, max_new_tokens=20)

# Decode and print output
output = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(output)

OutOfMemoryError: CUDA out of memory. Tried to allocate 270.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 111.12 MiB is free. Process 3592542 has 15.78 GiB memory in use. Of the allocated memory 15.51 GiB is allocated by PyTorch, and 17.49 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# from google.colab import userdata


# To use it make sure to add your Gemini api key in the colab secrets with `GOOGLE_API_KEY` as the variable name

# Load the language model
# model = ChatGoogleGenerativeAI(
#     model="gemini-2.0-flash-exp", temperature=0.5
# )

model = genai.GenerativeModel(model_name="gemini-1.5-pro")
# model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp")
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

genai.configure(api_key=google_api_key)


In [ ]:
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [ ]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("hf://datasets/Rajarshi-Roy-research/Som_bench_refcocog_refseg/data/train-00000-of-00001.parquet")

In [ ]:
df.head()

,id,image,obj_text,ref_ids,json_data
0,000000007601,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,[A black cow that is only half visible standin...,"[3, 1, 2]","[{'ref_id': 3, 'text': ['A black cow that is o..."
1,000000031187,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,[little girl in light green shirt; Little Girl...,"[2, 1, 3, 4]","[{'ref_id': 2, 'text': ['little girl in light ..."
2,000000066669,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,[an orange cat stuffed animal; The teddy bear ...,"[4, 5]","[{'ref_id': 4, 'text': ['an orange cat stuffed..."
3,000000017997,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,[The giraffe standing in the middle.; A giraff...,"[3, 2]","[{'ref_id': 3, 'text': ['The giraffe standing ..."
4,000000032707,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,[skier in orange coat in the right hand pictur...,"[2, 1]","[{'ref_id': 2, 'text': ['skier in orange coat ..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         100 non-null    object
 1   image      100 non-null    object
 2   obj_text   100 non-null    object
 3   ref_ids    100 non-null    object
 4   json_data  100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [ ]:
import re
import ast

def extract_list_from_string(text):
    """
    Extracts a list from a string by finding the first occurrence of square brackets.

    Parameters:
        text (str): The input string containing a Python list.

    Returns:
        list: Extracted list if found, otherwise an empty list.
    """
    match = re.search(r"\[.*?\]", text, re.DOTALL)
    if match:
        try:
            return ast.literal_eval(match.group(0))  # Safely parse the list
        except (SyntaxError, ValueError):
            return []
    return []

# Example usage
text = "```python\n[3, 1, 2]\n```"
print(extract_list_from_string(text))  # Output: [3, 1, 2]
print(type(extract_list_from_string(text)))  # Output: [3, 1, 2]


[3, 1, 2]
<class 'list'>


In [ ]:
import httpx
import base64




def get_prompt(obj_list):
  prompt=f"""I have labeled a bright numeric ID at the center for each visual object in the image. \nPlease tell me the IDs for:"""

  for i in obj_list:
    prompt += f"\n-{i}\n"

  prompt+= "\nPlease, properly give answer in the form of just a python list format accoriding to the order of objects asked\nAnswer Example:`[1,2,4..]`"

  # print(prompt)
  return prompt

def get_response(prompt,img_bytes):

  # Extract image data and encode to base64
  images_data = [
      {
          'mime_type': 'image/jpeg',
          'data': base64.b64encode(img_bytes).decode("utf-8")
      }
  ]

  # Generate response
  response = model.generate_content(images_data + [prompt])
  text = response.candidates[0].content.parts[0].text
  # print(text)

  return extract_list_from_string(text)

def get_prediction(index):

  img_bytes = df.loc[index,"image"]["bytes"]
  obj_list = df.loc[index,"obj_text"]

  prompt = get_prompt(obj_list)

  return get_response(prompt,img_bytes)


In [ ]:
get_prediction(1)

KeyboardInterrupt: 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         100 non-null    object
 1   image      100 non-null    object
 2   obj_text   100 non-null    object
 3   ref_ids    100 non-null    object
 4   json_data  100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [ ]:
from tqdm import tqdm
import time
error_indexes = []
error_titles = []

progress_bar = tqdm(total=len(df), desc="Processing")

# Wrap df.iterrows() with tqdm to show progress
for index, row in df.iterrows():
    # if pd.notna(row['image']):  # Check if 'image' is not NaN

    if pd.notna(row['image']) and pd.isna(row['gemini_ref_ids_pred']):  # Check if 'image' is not NaN

        if index % 2 == 0:  # Update tqdm every 100 iterations
          time.sleep(2)

        try:
          response = get_prediction(index)

          df.at[index, 'gemini_ref_ids_pred'] = f"{response}"

        except Exception as e:
          print(f"Error processing row {index}: {e}")
          error_indexes.append(index)
          error_titles.append(row['obj_text'])
          break

    progress_bar.update(1)


progress_bar.close()

Processing:  13%|█▎        | 13/100 [02:14<14:57, 10.32s/it]


KeyboardInterrupt: 

In [ ]:
get_prediction(20)

KeyboardInterrupt: 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   100 non-null    object
 1   image                100 non-null    object
 2   obj_text             100 non-null    object
 3   ref_ids              100 non-null    object
 4   json_data            100 non-null    object
 5   gemini_ref_ids_pred  17 non-null     object
dtypes: object(6)
memory usage: 4.8+ KB


In [ ]:
response

[45, 89, 45, 45, 45, 45, 15, 78]

In [ ]:
response

[3, 1, 2]

In [ ]:
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 932 entries, 0 to 931
Data columns (total 14 columns):
 #   Column                                                         Non-Null Count  Dtype 
---  ------                                                         --------------  ----- 
 0   section                                                        932 non-null    object
 1   title                                                          932 non-null    object
 2   abstract                                                       932 non-null    object
 3   url                                                            932 non-null    object
 4   Human story fetched                                            932 non-null    object
 5   AI generated article with context(Human Story)                 912 non-null    object
 6   AI generated story(without context)                            912 non-null    object
 7   Web retrieved URLs                                             932 non-